In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast
import sklearn
from sklearn.preprocessing import OneHotEncoder

In [2]:
# load and convert annotation data
path = "data/"
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
enc = MultiLabelBinarizer(classes = ["NORM", "MI", "STTC", "CD", "HYP"])

for i in range(1, 11):
    name = f"data/nparrays/labels/{str(i).zfill(2)}"
    Y_ = Y[(Y.strat_fold == i)].diagnostic_superclass.tolist()
    Y_ = enc.fit_transform(Y_)
    
    np.save(name, Y_)